In [3]:
import os
os.chdir('..')

In [4]:
from torchvision.datasets import MNIST
import torch
import torch.nn as nn
import torchvision.transforms as TF
from tqdm.auto import tqdm
from eXNN.NetBayesianization import wrap, api

In [5]:
train_ds = MNIST(root='./.cache', train=True, download=True, 
                 transform=TF.ToTensor()) 
test_ds = MNIST(root='./.cache', train=False, download=False, 
                transform=TF.ToTensor())

In [6]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=36, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=36, shuffle=False)

In [7]:
num_classes = 10

In [8]:
model = nn.Sequential(nn.Flatten() ,nn.Linear(28*28, 128), 
                       nn.ReLU(), nn.Linear(128, 64), 
                       nn.ReLU(), nn.Linear(64, num_classes), nn.Softmax(dim=1))
optimizer = torch.optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
criterion = nn.CrossEntropyLoss()
images, labels = next(iter(train_dl))
images = images.view(images.shape[0], -1)
logps = model(images)
loss = criterion(logps, labels)

In [9]:
# train
n_epochs = 20
for e in range(n_epochs):
    running_loss = 0
    for images, labels in train_dl:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_dl)))

Epoch 0 - Training loss: 2.261323278294971
Epoch 1 - Training loss: 1.805009106139473
Epoch 2 - Training loss: 1.6562042623919215
Epoch 3 - Training loss: 1.6340304391428462
Epoch 4 - Training loss: 1.6253673680613838
Epoch 5 - Training loss: 1.6201491944433761
Epoch 6 - Training loss: 1.6161638898054282
Epoch 7 - Training loss: 1.6131943888341014
Epoch 8 - Training loss: 1.610495502461054
Epoch 9 - Training loss: 1.6082047563723338
Epoch 10 - Training loss: 1.6061707546987
Epoch 11 - Training loss: 1.6044750517545951
Epoch 12 - Training loss: 1.6026566071501733
Epoch 13 - Training loss: 1.6009767129168084
Epoch 14 - Training loss: 1.5995301769104415
Epoch 15 - Training loss: 1.598026476223882
Epoch 16 - Training loss: 1.596490390156298
Epoch 17 - Training loss: 1.5951921355602765
Epoch 18 - Training loss: 1.5939387459917989
Epoch 19 - Training loss: 1.592780917435974


In [10]:
correct_count, all_count = 0, 0
for images,labels in test_dl:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.8676


In [11]:
# build classical bayesian model
bayes_model = api.BasicBayesianWrapper(model, 'basic', 0.1, None, None)

In [12]:
# predict
n_iter = 5
bayes_model.predict(images, n_iter)

{'mean': tensor([[2.9532e-11, 5.2978e-16, 8.1444e-11, 1.7176e-14, 9.9914e-01, 1.0547e-10,
          4.2251e-07, 3.6712e-07, 1.2748e-06, 8.5830e-04],
         [1.1691e-05, 4.5427e-19, 1.2401e-11, 2.8679e-11, 6.7911e-02, 8.0002e-10,
          8.9902e-11, 6.6425e-03, 2.2742e-02, 9.0269e-01],
         [3.4553e-11, 2.5543e-14, 1.0935e-09, 5.7184e-15, 9.9760e-01, 4.0230e-10,
          1.0476e-06, 2.0018e-06, 4.5529e-07, 2.3946e-03],
         [1.1240e-07, 1.3264e-07, 4.9493e-02, 9.4469e-01, 2.1763e-11, 7.1732e-09,
          5.9953e-12, 1.9604e-07, 5.8205e-03, 6.3144e-09],
         [6.4917e-07, 1.7414e-11, 6.5679e-06, 3.0981e-14, 3.3005e-05, 3.6413e-10,
          9.9993e-01, 4.1751e-09, 2.9818e-05, 8.0287e-07],
         [6.5410e-11, 6.6342e-12, 1.3535e-09, 2.4292e-14, 9.9905e-01, 1.4828e-09,
          8.2082e-06, 1.3887e-04, 6.4032e-06, 7.9413e-04],
         [3.4536e-11, 9.9948e-01, 4.6569e-05, 2.0753e-05, 1.9267e-09, 3.0979e-08,
          1.0521e-10, 1.5474e-04, 2.9737e-04, 8.0818e-08],
     

In [13]:
# build bayesian model with beta distibution
bayes_model = api.BasicBayesianWrapper(model, 'beta', None, 0.6, 0.3)

In [14]:
# predict
n_iter = 5
bayes_model.predict(images, n_iter)

{'mean': tensor([[0.0441, 0.0519, 0.0385, 0.0234, 0.6511, 0.0391, 0.0440, 0.0502, 0.0187,
          0.0391],
         [0.0441, 0.0520, 0.0385, 0.0234, 0.1724, 0.0391, 0.0441, 0.0503, 0.0183,
          0.5178],
         [0.0436, 0.0514, 0.0381, 0.0233, 0.6506, 0.0387, 0.0436, 0.0497, 0.0225,
          0.0387],
         [0.0441, 0.0519, 0.0511, 0.6107, 0.0511, 0.0391, 0.0440, 0.0502, 0.0188,
          0.0391],
         [0.0437, 0.0514, 0.0381, 0.0233, 0.0584, 0.0387, 0.6343, 0.0497, 0.0225,
          0.0400],
         [0.0436, 0.0514, 0.0381, 0.0233, 0.6506, 0.0387, 0.0436, 0.0497, 0.0225,
          0.0387],
         [0.0441, 0.6481, 0.0417, 0.0237, 0.0511, 0.0391, 0.0440, 0.0503, 0.0189,
          0.0391],
         [0.0441, 0.0520, 0.0385, 0.0234, 0.0512, 0.0391, 0.0441, 0.4600, 0.0182,
          0.2293],
         [0.0441, 0.0520, 0.4107, 0.2512, 0.0512, 0.0391, 0.0441, 0.0503, 0.0182,
          0.0391],
         [0.0436, 0.0514, 0.0409, 0.0233, 0.0506, 0.0387, 0.6407, 0.0497, 0.0225,
 